# Evaluation of Multi-Agent approach using an orchestrator

In [1]:
import sys
import os
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/")
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/agents")
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/prompt_templates")
from dotenv import load_dotenv

sys.path.insert(0, os.path.abspath("../src"))
print(sys.path)

load_dotenv()
from approach_evaluator import ApproachEvaluator
from orchestrator_agent import create_sql_from_natural_text, visualize_graph
visualize_graph()

['/Users/yannikhahn/Code/LLMsAgents-TextToSQL/src', '/Users/yannikhahn/Code/LLMsAgents-TextToSQL/evaluation/src', '/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/yannikhahn/Library/Caches/pypoetry/virtualenvs/llmsagents-texttosql-JgGAmzxu-py3.12/lib/python3.12/site-packages', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/agents', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/prompt_templates']


ModuleNotFoundError: No module named 'orchestrator_agent'

In [2]:
import pandas as pd

data_dir = os.getenv("DATA_DIRECTORY")
sql_input_file = pd.read_csv("../sample/dev.csv").sample(5)

# Config is just for logging purposes and does not affect the run
run_config = {
    "k_shot_prompting_creator_agent": 2,
    "k_shot_prompting_fixer_agent": 1,
    "k_shot_prompting_feedback_agent": 2,
    "k_shot_prompting_select_relevant_database": 0,
    "k_shot_prompting_select_relevant_tables": 2
}

evaluator = ApproachEvaluator(
    run_config=run_config,
    question_ids=sql_input_file["question_id"],
    approach=create_sql_from_natural_text
)
evaluator.evaluate()

Input text: Of the first 100 molecules in number order, how many are carcinogenic?

Predicted result: [(0,)], Target result: [(29,)]
Input text: Which post by Harvey Motulsky has the most views? Please give the id and title of this post.

Predicted result: [(109, "Power of Holm's multiple comparison testing compared to others")], Target result: [(109, "Power of Holm's multiple comparison testing compared to others")]
Input text: Give the race of the blue-haired male superhero.

Orchestrator Agent: validate: superhero
Orhcestrator Agent: validate: SELECT race.race FROM superhero 
        JOIN colour ON superhero.hair_colour_id = colour.id
        JOIN race ON superhero.race_id = race.id
        WHERE superhero.gender_id = gender.id AND superhero.eye_colour_id = colour.id AND superhero.hair_colour_id = colour.id AND superhero.skin_colour_id = colour.id AND superhero.publisher_id = publisher.id AND superhero.alignment_id = alignment.id AND superhero.height_cm = height_cm.id AND superhero.

{'question_id': [678, 817, 938, 893, 204],
 'predicted_query': ["SELECT COUNT(label) FROM molecule WHERE molecule_id <= 100 AND label = 'carcinogenic';",
  "SELECT posts.Id, posts.Title FROM posts INNER JOIN users ON posts.OwnerUserId = users.Id WHERE users.DisplayName = 'Harvey Motulsky' ORDER BY posts.ViewCount DESC LIMIT 1;",
  '',
  'SELECT drivers.forename || \' \' || drivers.surname AS "Name", driverStandings.points AS "Points" FROM driverStandings INNER JOIN drivers ON driverStandings.driverId = drivers.driverId WHERE driverStandings.raceId = (SELECT raceId FROM races WHERE year = 2017 AND name = \'Chinese Grand Prix\') ORDER BY driverStandings.points DESC LIMIT 3;',
  "SELECT drivers.forename, drivers.surname, drivers.url FROM drivers\n        JOIN results ON drivers.driverId = results.driverId\n        JOIN races ON results.raceId = races.raceId\n        WHERE races.year = 2008 AND races.name = 'Australian Grand Prix' AND results.position = 1;"],
 'predicted_database': ['toxic

In [2]:
import pandas as pd
run = pd.read_csv("./results/results_full_run_291124/results.csv")

run.head()

,input_texts,predicted_query,predicted_database,target_queries,is_correct
0,Among the cards with converted mana cost highe...,SELECT COUNT(*) FROM cards WHERE convertedMana...,card_games,SELECT SUM(CASE WHEN T1.power LIKE '*' OR T1.p...,0
1,Calculate the average home team goal in the 20...,SELECT AVG(home_team_goal) FROM Match WHERE se...,european_football_2,SELECT CAST(SUM(t2.home_team_goal) AS REAL) / ...,1
2,Which college do most of the members go to?,SQL\r\n\r\nSELECT college\r\nFROM major\r\nWHE...,student_club,SELECT T2.college FROM member AS T1 INNER JOIN...,0
3,Calculate the percentage of molecules containi...,SELECT (CAST(COUNT(DISTINCT(molecule_id)) AS R...,toxicology,SELECT CAST(COUNT(CASE WHEN T1.element = 'h' A...,0
4,"Provide the related post title of ""How to tell...","SELECT ""Post Title"" FROM 'thrombosis_predictio...",thrombosis_prediction,SELECT T3.Title FROM postLinks AS T1 INNER JOI...,0


In [3]:
from approach_evaluator import valid_efficiency_score

ves = valid_efficiency_score(run)

print(f"Valid Efficiency Score: {ves}")

Valid Efficiency Score: 0.19128302335913877
